### Importacion de librerias 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns

### Extraer datos

In [14]:
df = pd.read_csv("../data/clean_dataset.csv")
df

,Freedom,Happiness_Rank,Perceptions_of_corruption,Happiness_Score,Social_support,Year,Economic_Health_Index,region_Australia and New Zealand,region_Central and Eastern Europe,region_Eastern Asia,region_Latin America and Caribbean,region_Middle East and Northern Africa,region_North America,region_Southeastern Asia,region_Southern Asia,region_Sub-Saharan Africa,region_Western Europe
0,0.66557,1,0.41978,7.587,1.34951,2015,1.772106,0,0,0,0,0,0,0,0,0,1
1,0.62877,2,0.14145,7.561,1.40223,2015,1.626761,0,0,0,0,0,0,0,0,0,1
2,0.64938,3,0.48357,7.527,1.36058,2015,1.458406,0,0,0,0,0,0,0,0,0,1
3,0.66973,4,0.36503,7.522,1.33095,2015,1.720453,0,0,0,0,0,0,0,0,0,1
4,0.63297,5,0.32957,7.427,1.32261,2015,1.548120,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,0.55500,152,0.41100,3.334,0.71100,2019,-0.963090,0,0,0,0,0,0,0,0,1,0
778,0.41700,153,0.14700,3.231,0.88500,2019,-1.087552,0,0,0,0,0,0,0,0,1,0
779,0.00000,154,0.02500,3.203,0.51700,2019,-1.699650,0,0,0,0,0,0,0,1,0,0
780,0.22500,155,0.03500,3.083,0.00000,2019,-2.991922,0,0,0,0,0,0,0,0,1,0


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# Variables predictoras y objetivo
X = df.drop(columns=['Happiness_Score'])
y = df['Happiness_Score']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Modelos de regresión
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.01),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(objective='reg:squarederror', random_state=42)
}

# Evaluar modelos
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        'Modelo': name,
        'R² Score': round(r2_score(y_test, y_pred), 4)
    })

# Mostrar resultados
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R² Score', ascending=False).reset_index(drop=True)

print("\nResultados de los Modelos:")
print(results_df.to_string(index=False))



Resultados de los Modelos:
           Modelo  R² Score
Gradient Boosting    0.9971
    Random Forest    0.9971
          XGBoost    0.9968
 Ridge Regression    0.9860
Linear Regression    0.9859
 Lasso Regression    0.9856


In [18]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Configuración de validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Resultados de métricas
results = []

for name, model in models.items():
    # R² por validación cruzada
    r2_scores = cross_val_score(model, X, y, scoring='r2', cv=cv)
    
    # Ajustar y predecir en un fold de ejemplo (train/test split)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))


    results.append({
        'Modelo': name,
        'R² CV Mean': round(np.mean(r2_scores), 4),
        'R² CV Std': round(np.std(r2_scores), 4),
        'MAE': round(mae, 4),
        'RMSE': round(rmse, 4)
    })

# Mostrar resultados
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R² CV Mean', ascending=False).reset_index(drop=True)

print("\nResultados con Validación Cruzada (5 folds):")
print(results_df.to_string(index=False))



Resultados con Validación Cruzada (5 folds):
           Modelo  R² CV Mean  R² CV Std    MAE   RMSE
Gradient Boosting      0.9969     0.0005 0.0469 0.0598
    Random Forest      0.9968     0.0007 0.0465 0.0594
          XGBoost      0.9968     0.0006 0.0443 0.0624
 Ridge Regression      0.9862     0.0020 0.0973 0.1306
Linear Regression      0.9861     0.0020 0.0979 0.1312
 Lasso Regression      0.9841     0.0020 0.1015 0.1325
